#### 旋转位置编码

In [ ]:
import torch

def sinusoidal_pos_emedbedding(bs, seq_len, nums_head, dmodel):
    position = torch.arange(0, seq_len, dtype = torch.float).unsqueeze(-1)## [seq_len, 1]
    ids = torch.arange(0, dmodel//2, dtype = torch.float) ##公式中的i
    theta = torch.pow(10000, -2 * ids / dmodel) ##公式中的theta,和每个token维度有关
    embeddings = position * theta ## [seq_len, dmodel//2]   对应论文公式中的sin(θ * i)
    embeddings = torch.stack([torch.sin(embeddings), torch.cos(embeddings)], dim=-1) ## [seq_len, dmodel//2, 2]
    embeddings = embeddings.unsqueeze(0).repeat(bs, nums_head, 1, 1, 1) ## [bs, nums_head, seq_len, dmodel//2, 2] 在bs和head维度上进行重复
    embeddings = torch.reshape(embeddings, (bs, nums_head, seq_len, dmodel))## [bs, nums_head, seq_len, dmodel],恢复为原始维度，在最后一个维度，偶数sin，奇数cos
    return embeddings


def RoPE(q, k):
    bs = q.size(0)
    seq_len = q.size(1)
    nums_head = q.size(2)
    dmodel = q.size(3)
    pos_emb = sinusoidal_pos_emedbedding(bs, seq_len, nums_head, dmodel)
    
    cos_pos = pos_emb[..., 1::2].repeat_interleave(2, dim=-1) ## [bs, nums_head, seq_len, dmodel]奇数列拿来复制
    sin_pos = pos_emb[..., ::2].repeat_interleave(2, dim=-1) ## [bs, nums_head, seq_len, dmodel]偶数列
    
    q2 = torch.stack(-q[..., 1::2], q[..., ::2], dim=-1)#正负交替（对于sin）
    q2 = q2.reshape(q.shape)
    q = q * cos_pos + q2 * sin_pos # [bs, nums_head, seq_len, dmodel]
    
    
    k2 = torch.stack([-k[..., 1::2], k[..., ::2]], dim=-1)
    k2 = k2.reshape(k.shape)
    # 更新kw, *对应位置相乘
    k = k * cos_pos + k2 * sin_pos # [bs, nums_head, seq_len, dmodel]
    
    return q, k

In [26]:
position = torch.arange(0, 5, dtype = torch.float).unsqueeze(-1)
ids = torch.arange(0, 10//2, dtype = torch.float)
theta = torch.pow(10000, -2 * ids / 10)
embeddings = position * theta
embeddings = torch.stack([torch.sin(embeddings), torch.cos(embeddings)], dim=-1)
# embeddings = embeddings.repeat((8, 4, *([1] * len(embeddings.shape)))) 
embeddings = embeddings.unsqueeze(0).repeat(8, 4, 1, 1,1)
print(embeddings.shape)
embeddings = torch.reshape(embeddings, (8, 4, 5, 10))
print(embeddings.shape)

torch.Size([8, 4, 5, 5, 2])
torch.Size([8, 4, 5, 10])
